In [1]:
VER = 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd, numpy as np
import pickle, glob, gc

from collections import Counter
import itertools

# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test(files):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(files)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [31]:
train = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_parquet/*')
print('train data has shape',train.shape)

train data has shape (7683577, 4)


In [32]:
def statistics_process(df):
    df = df.groupby('session').agg({'session':'count',
                                    'aid':['count','nunique'],
                                    'ts':['min','max','sum','mean','std'],
                                    'type':['min','max','sum','mean','std']})
    df.columns = ['user_user_count',
                  'user_item_count','user_nunique',
                  'user_ts_min','user_ts_max','user_ts_sum','user_ts_mean','user_ts_std',
                  'user_type_min','user_type,max','user_type_sum','user_type_mean','user_type_std']
    return df# sum,mean,min,max,count,std,nunique,cumcount

In [33]:
train = statistics_process(train)
train.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/user/val_user{VER}.pqt')

In [34]:
train.head()

,user_user_count,user_item_count,user_nunique,user_ts_min,user_ts_max,user_ts_sum,user_ts_mean,user_ts_std,user_type_min,"user_type,max",user_type_sum,user_type_mean,user_type_std
session,,,,,,,,,,,,,
11098528,1,1,1,1661119200,1661119200,1.661119e+09,1.661119e+09,NaN,0,0,0.0,0.000000,NaN
11098529,1,1,1,1661119200,1661119200,1.661119e+09,1.661119e+09,NaN,0,0,0.0,0.000000,NaN
11098530,6,6,2,1661119200,1661120532,9.966718e+09,1.661120e+09,547.326380,0,1,1.0,0.166667,0.408248
11098531,24,24,11,1661119200,1661119746,3.986687e+10,1.661119e+09,174.236949,0,2,8.0,0.333333,0.761387
11098532,2,2,2,1661119201,1661119996,3.322239e+09,1.661120e+09,562.149891,0,0,0.0,0.000000,0.000000


In [35]:
del train
_ = gc.collect()

In [36]:
test = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/original/test_parquet/*')
print('Test data has shape',test.shape)

Test data has shape (6928123, 4)


In [37]:
test = statistics_process(test)
test.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/user/test_user{VER}.pqt')